<a href="https://colab.research.google.com/github/VictoriaOAbel/Analyses/blob/main/Amazon_Customers_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

How to Read Data from SQLite Database

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3

In [ ]:
sqlite3.connect('amazon.db')

In [ ]:
con = sqlite3.connect(r'/content/database.sqlite')

In [ ]:
type(con)

In [ ]:
df = pd.read_sql_query("SELECT * FROM REVIEWS" , con)

In [ ]:
df.shape

Data Preparation

In [ ]:
type(df)

In [ ]:
df.head(4)

In [ ]:
df.columns

In [ ]:
# Invalid row (not true)
df['HelpfulnessNumerator'] > df['HelpfulnessDenominator']

In [ ]:
df[df['HelpfulnessNumerator'] > df['HelpfulnessDenominator']]

In [ ]:
df_valid = df[df['HelpfulnessNumerator'] <= df['HelpfulnessDenominator']]

The code snippet you've provided filters a DataFrame to include only rows where the value of `HelpfulnessNumerator` is greater than the value of `HelpfulnessDenominator`. Here’s how you can do it in Python using pandas:

```python
# Assuming df is your DataFrame
filtered_df = df[df['HelpfulnessNumerator'] > df['HelpfulnessDenominator']]
print(filtered_df)
```

This code creates a new DataFrame (`filtered_df`) containing only the rows where `HelpfulnessNumerator` is greater than `HelpfulnessDenom

In [ ]:
df_valid.shape

In [ ]:
df_valid.columns

In [ ]:
df_valid.duplicated(['UserId', 'ProfileName', 'Time', 'Text'])

In [ ]:
df_valid[df_valid.duplicated(['UserId', 'ProfileName', 'Time', 'Text'])]

In [ ]:
data = df_valid.drop_duplicates(subset=['UserId', 'ProfileName', 'Time', 'Text'])

# data = df_valid.drop_duplicates(subset=['UserId', 'ProfileName', 'Time', 'Text'], inplace=True)

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data['Time']

In [ ]:
pd.to_datetime(data['Time'])

In [ ]:
data['Time'] = pd.to_datetime(data['Time'], unit='s')

In [ ]:
import warnings
from warnings import filterwarnings
filterwarnings('ignore')
# warnings.filterwarnings('ignore')

In [ ]:
data['Time'] = pd.to_datetime(data['Time'], unit='s')

How Amazon Recommend Products

In [ ]:
# Analyse to what user Amazon can recommend more products

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data['ProfileName']

In [ ]:
data['ProfileName'].unique

In [ ]:
# Any unique functions, returns count
data['ProfileName'].nunique

In [ ]:
data['UserId'].nunique

In [ ]:
data.columns

In [ ]:
data.groupby('UserId').agg({'Summary': 'count', 'Text': 'count', 'Score': 'mean','ProductId': 'count'})

In [ ]:
recommend_df = data.groupby('UserId').agg({'Summary': 'count', 'Text': 'count', 'Score': 'mean','ProductId': 'count'}).sort_values(by='ProductId', ascending=False)

In [ ]:
recommend_df

In [ ]:
recommend_df.columns = ['Number_of_Summary', 'Number_of_Text', 'Average_Score', 'Number_of_Products']

In [ ]:
recommend_df

In [ ]:
recommend_df.index

In [ ]:
recommend_df.index[0:10]

In [ ]:
recommend_df['Number_of_Products'][0:10]

In [ ]:
recommend_df['Number_of_Products'][0:10].values

In [ ]:
plt.bar(recommend_df.index[0:10], recommend_df['Number_of_Products'][0:10].values)
plt.xticks(rotation='vertical')

In [ ]:
# plt.bar(recommend_df.index[0:10], recommend_df['Number_of_Products'][0:10].values)
# plt.xticks(rotation=90)
# plt.show()

Analysing which Product has good number of reviews

In [ ]:
# Which product has good number of reviews

In [ ]:
data.columns

In [ ]:
data['ProductId']

In [ ]:
# data['ProductId'].value_counts()

In [ ]:
data['ProductId'].unique()

In [ ]:
len(data['ProductId'].unique())

In [ ]:
# Most frequent sold products with high number of reviews

In [ ]:
prod_count = data['ProductId'].value_counts().to_frame()

In [ ]:
prod_count

In [ ]:
# Only consider products sold more than 500 times
prod_count['ProductId'] > 500

In [ ]:
prod_count[prod_count['ProductId'] > 500]

In [ ]:
prod_count[prod_count['ProductId'] > 500].index

In [ ]:
freq_prod_ids = prod_count[prod_count['ProductId'] > 500].index

In [ ]:
freq_prod_ids

The code snippet you provided counts the occurrences of each unique value in the `ProductId` column of the `data` DataFrame and converts the result into a new DataFrame called `prod_count`. Here’s the complete code with an explanation:

```python
# Counting the occurrences of each unique value in the 'ProductId' column
prod_count = data['ProductId'].value_counts().to_frame()

# Display the resulting DataFrame
print(prod_count)
```

In [ ]:
data['ProductId'].isin(freq_prod_ids)

In [ ]:
fre_prod_df = data[data['ProductId'].isin(freq_prod_ids)]

In [ ]:
fre_prod_df

In [ ]:
fre_prod_df.columns

In [ ]:
sns.countplot(x='ProductId', data=fre_prod_df)

In [ ]:
sns.countplot(y='ProductId', data=fre_prod_df)

In [ ]:
sns.countplot(y='ProductId', data=fre_prod_df, hue='Score')

Understanding Behaviours of Amazon Users

In [1]:
# Is there any difference between behaviour of frequent and non-frequent visitors
data.columns

In [ ]:
x = data['UserId'].value_counts()
x

In [ ]:
data.head(7)

In [ ]:
data['UserId']

In [ ]:
x['AY12DBB0U420B']

In [ ]:
data['viewer_type'] = data['UserId'].apply(lambda user : "Frequent" if x[user]>50 else "Not Frequent")

In [ ]:
data.head(3)

In [ ]:
data['viewer_type'].unique( )

In [ ]:
# Creating Non-Frequent views dataframe
data['viewer_type'] == 'Not Frequent'

# Frequent views dataframe
# data['viewer_type'] == 'Frequent'

In [ ]:
not_freq_df = data[data['viewer_type'] == 'Not Frequent']
freq_df = data[data['viewer_type'] == 'Frequent']

In [ ]:
freq_df['Score'].value_counts()
# freq_df['Score'].value_counts(normalize=True)

In [ ]:
#To convert to percentage
freq_df['Score'].value_counts()/len(freq_df)*100

In [ ]:
not_freq_df['Score'].value_counts()/len(not_freq_df)*100

In [ ]:
freq_df['Score'].value_counts().plot()

In [ ]:
freq_df['Score'].value_counts().plot(kind='bar')

In [ ]:
not_freq_df['Score'].value_counts().plot(kind='bar')

Analysing Your Frequent Users

In [ ]:
# Are frequent users more verbose
data.columns

In [ ]:
data['Text']

In [ ]:
data[['UserId', 'ProductId', 'Text']]

In [ ]:
data['Text']

In [ ]:
data['Text'][0]

In [ ]:
type(data['Text'][0])

In [ ]:
type(data['Text'][0].split(' '))

In [ ]:
data['Text'][0].split(' ')

In [ ]:
len(data['Text'][0].split(' '))

In [ ]:
# return len(text)
def calculate_len(text):
  return len(text.split(' '))

# OR

# def calculate_length(text):
#     len(text.split(' '))

In [ ]:
data['Text_length'] = data['Text'].apply(calculate_length)

In [ ]:
data['viewer_type'].unique()

In [ ]:
not_freq_data = data[data['viewer_type'] == 'Not Frequent']
freq_data = data[data['viewer_type'] == 'Frequent']

In [ ]:
fig = plt.figure
ax1 = fig.add_subplot(1,2,1)      # You can remove commas, no space i.e. 121 or 122
ax1.boxplot(freq_data['Text_length'])
ax1.set_xlabel('Frequency of frequent reviewers')
ax1.set_xlim(0,600)

ax2 = fig.add_subplot(1,2,2)
ax2.boxplot(not_freq_data['Text_length'])
ax2.set_xlabel('Frequency of not-frequent reviewers')
ax2.set_ylim(0,600)

Perform Sentiment Analysis on Data

In [ ]:
# Sentiment Analysis
!pip install textblob

In [ ]:
from textblob import TextBlob

In [ ]:
data['Summary']

In [ ]:
data['Summary'][0]

In [ ]:
TextBlob('Good Quality Dog Food')

In [ ]:
TextBlob('Good Quality Dog Food').sentiment

In [ ]:
TextBlob('Good Quality Dog Food').sentiment.polarity

In [ ]:
data.shape

In [ ]:
sample = data[0:50000]

In [ ]:
polarity = []           # Create blank list

for text in sample['Summary']:
    try:
        polarity.append(TextBlob(text).sentiment.polarity)
    except:
        polarity.append(0)


In [ ]:
len(polarity)

In [ ]:
sample['polarity'] = polarity

In [ ]:
sample.head()

In [ ]:
# Negative and Positive polarity

sample_negative = sample['polarity']<0        # Negative polarity
sample_positive = sample['polarity']>0        # Positive polarity

In [ ]:
sample_negative['Summary']

In [ ]:
from collections import Counter

In [ ]:
Counter(sample_negative['Summary'])

In [ ]:
Counter(sample_negative['Summary']).most_common(10)

In [ ]:
Counter(sample_positive['Summary'])

In [ ]:
Counter(sample_positive['Summary']).most_common(10)